## Import

In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from tqdm import tqdm
from functools import reduce

import numpy as np
import pandas as pd
import re, time
import requests
import pickle
import warnings

warnings.filterwarnings(action='ignore')

In [2]:
import os
os.getcwd()

'C:\\Users\\PC0\\Documents\\GitHub\\Stock-price-prediction\\ipynb\\crawling-code'

In [3]:
out_data_path = 'C:\\Users\\PC0\\Documents\\datasets\\stock_price\\'
data_path = 'C:\\Users\\PC0\\Documents\\GitHub\\Stock-price-prediction\\datasets\\'

* crawling

In [10]:
options = webdriver.ChromeOptions()

options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.82 Safari/537.36")

driver = webdriver.Chrome("C:\\Users\\PC0\\Desktop\\chromedriver.exe", options=options)

* agricultural 

In [11]:
start_day = "2012/01/01"

In [12]:
cont = ['germany', 'u.s.', 'brazil', 'switzerland', 'uk', 'italy', 'japan',
       'china', 'canada', 'france', 'south-korea', 'australia']

In [13]:
urls = ['https://kr.investing.com/rates-bonds/{}-1-year-bond-yield-historical-data'.format(x) for x in cont]
urls.extend(['https://kr.investing.com/rates-bonds/{}-10-year-bond-yield-historical-data'.format(x) for x in cont])

In [14]:
names = [(x.split("/")[-1]).split("-year")[0] + "_bond" for x in urls]
names = [x.split("_bond")[0] for x in names]

In [15]:
start_idx = 0

In [17]:
# inner loop
index_list = []

for idx, url in tqdm(enumerate(urls[start_idx:])):
    driver.get(url)
    time.sleep(5)

    # 날짜 입력란
    tmp_date = driver.find_element_by_css_selector("div#widgetFieldDateRange")
    tmp_date.click()
    driver.implicitly_wait(3)

    tmp_input = driver.find_element_by_css_selector("input#startDate")
    time.sleep(0.5)

    # 기존 날짜 지우기
    for _ in range(11) :
        tmp_input.send_keys(Keys.BACKSPACE)
        tmp_input = driver.find_element_by_css_selector("input#startDate")
        time.sleep(0.5)

    tmp_input.send_keys(start_day)

    # 요청 버튼
    request_btn = driver.find_element_by_css_selector("a#applyBtn")
    request_btn.click()
    time.sleep(5)

    tables = driver.find_elements_by_css_selector("table#curr_table")

    values = tables[0].text.split("\n")[1:]

    values = [re.sub("[^0-9a-zA-Z.\\- ]","",x) for x in values]

    name = names[idx]

    index_dict = {"date":[],
                 "{}_close".format(name):[],
                 "{}_var".format(name):[]}

    for val in values:
        try :
            year, month, day, close, _, _, _, variation= val.split(" ")
        except ValueError:
            year, month, day, close, _, _, _, vol, variation= val.split(" ")

        date = "-".join([year, month, day])
        index_dict['date'].append(date)
        index_dict['{}_close'.format(name)].append(close)
        index_dict['{}_var'.format(name)].append(variation)

    index_list.append(pd.DataFrame.from_dict(index_dict, orient = "index").T)
    start_idx += 1

index_df = reduce(lambda  left,right: pd.merge(left,right,on=['date'], how='outer'), index_list)

2it [01:11, 35.66s/it]


ElementClickInterceptedException: Message: element click intercepted: Element <a href="javascript:void(0)" id="applyBtn" class="newBtn Arrow LightGray float_lang_base_2" data-handler="hide" data-event="click">...</a> is not clickable at point (937, 638). Other element would receive the click: <div class="right">...</div>
  (Session info: chrome=98.0.4758.102)
Stacktrace:
Backtrace:
	Ordinal0 [0x00D2A0A3+2597027]
	Ordinal0 [0x00CBBA61+2144865]
	Ordinal0 [0x00BB3C58+1064024]
	Ordinal0 [0x00BE663F+1271359]
	Ordinal0 [0x00BE4887+1263751]
	Ordinal0 [0x00BE24DB+1254619]
	Ordinal0 [0x00BE12C9+1249993]
	Ordinal0 [0x00BD6EA3+1207971]
	Ordinal0 [0x00BF87AC+1345452]
	Ordinal0 [0x00BD6874+1206388]
	Ordinal0 [0x00BF8904+1345796]
	Ordinal0 [0x00C08B1A+1411866]
	Ordinal0 [0x00BF85C6+1344966]
	Ordinal0 [0x00BD54C6+1201350]
	Ordinal0 [0x00BD63B6+1205174]
	GetHandleVerifier [0x00ED10A2+1681842]
	GetHandleVerifier [0x00F83B0C+2413596]
	GetHandleVerifier [0x00DC1221+568113]
	GetHandleVerifier [0x00DBFFA3+

In [ ]:
index_df = index_df.sort_values("date", ascending = False)

In [ ]:
index_df.tail()

In [60]:
index_df.to_csv(data_path + "bond_df.csv", index = False)